In [1]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)

In [3]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(6366, 6)


In [4]:
#printSchema
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [5]:
#view the dataset
df.show(5)

+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [7]:
#Exploratory Data Analysis
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [8]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [9]:
df.groupBy('rate_marriage').count().show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [10]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [11]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [12]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [11]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [15]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [16]:
#select data for building model
model_df=df.select(['features','affairs'])

In [17]:
train_df,test_df=model_df.randomSplit([0.75,0.25])

In [18]:
train_df.count()

4800

In [19]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1574|
|      0| 3226|
+-------+-----+



In [20]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  479|
|      0| 1087|
+-------+-----+



In [21]:
from pyspark.ml.classification import RandomForestClassifier

In [22]:
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)

In [23]:
rf_predictions=rf_classifier.transform(test_df)

In [24]:
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[18.7524598757082...|[0.37504919751416...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[21.0021478623554...|[0.42004295724710...|       1.0|
|[1.0,27.0,6.0,0.0...|      0|[20.1166611727778...|[0.40233322345555...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[19.0905206218080...|[0.38181041243616...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[16.3348579592130...|[0.32669715918426...|       1.0|
|[1.0,27.0,9.0,4.0...|      0|[13.3128973003485...|[0.26625794600697...|       1.0|
|[1.0,32.0,13.0,0....|      1|[16.7812008990910...|[0.33562401798182...|       1.0|
|[1.0,32.0,13.0,2....|      1|[12.6966189294366...|[0.25393237858873...|       1.0|
|[1.0,32.0,13.0,2....|      1|[12.6766379097881...|[0.25353275819576...|    

In [25]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1261|
|       1.0|  305|
+----------+-----+



In [26]:
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |affairs|prediction|
+----------------------------------------+-------+----------+
|[0.37504919751416455,0.6249508024858356]|1      |1.0       |
|[0.42004295724710805,0.579957042752892] |1      |1.0       |
|[0.40233322345555694,0.597666776544443] |0      |1.0       |
|[0.3818104124361619,0.6181895875638381] |1      |1.0       |
|[0.32669715918426007,0.6733028408157399]|0      |1.0       |
|[0.26625794600697006,0.7337420539930299]|0      |1.0       |
|[0.3356240179818214,0.6643759820181787] |1      |1.0       |
|[0.25393237858873335,0.7460676214112667]|1      |1.0       |
|[0.2535327581957624,0.7464672418042376] |1      |1.0       |
|[0.3319327677415531,0.6680672322584469] |1      |1.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [30]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)

In [31]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 73%


In [32]:
print(rf_accuracy)

0.7279693486590039


In [33]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)

In [34]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 71%


In [35]:
rf_precision

0.7085017563673452

In [37]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)

In [38]:
print(rf_auc)

0.7421702296835062


In [39]:
# Feature importance

In [40]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.5536, 1: 0.0364, 2: 0.2358, 3: 0.0803, 4: 0.0939})

In [41]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [49]:
# Save the model 

In [50]:
pwd

'/home/jovyan/work'

In [52]:
rf_classifier.save("/home/jovyan/work/RF_model")

In [54]:
from pyspark.ml.classification import RandomForestClassificationModel

In [55]:
rf=RandomForestClassificationModel.load("/home/jovyan/work/RF_model")

In [58]:
model_preditions=rf.transform(test_df)

In [60]:
model_preditions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[188.676639360932...|[0.37735327872186...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[195.425833792250...|[0.39085166758450...|       1.0|
|[1.0,27.0,6.0,0.0...|      0|[193.138478579040...|[0.38627695715808...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[185.424877645536...|[0.37084975529107...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[164.685852316351...|[0.32937170463270...|       1.0|
|[1.0,27.0,9.0,4.0...|      0|[142.006095001922...|[0.28401219000384...|       1.0|
|[1.0,32.0,13.0,0....|      1|[176.885399312490...|[0.35377079862498...|       1.0|
|[1.0,32.0,13.0,2....|      1|[128.585405941664...|[0.25717081188332...|       1.0|
|[1.0,32.0,13.0,2....|      1|[126.963464206019...|[0.25392692841203...|    